In [55]:
%load_ext autoreload
%autoreload 2
import glob, os

#os.environ["CUDA_VISIBLE_DEVICES"]="0"  #uncomment for GPU use

%matplotlib inline
import numpy as np
import IPython
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib 
import matplotlib.tri as tri

import tensorflow as tf
import mdn
from tensorflow import keras

from matplotlib.ticker import FormatStrFormatter
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import binned_statistic
import math as m
from joblib import Parallel, delayed, dump, load
from keras.callbacks import History 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
pathSave = "."

In [57]:
class getData:
    def __init__(self, _paraVecIndices, _xparaVecIndices, _tProfBounds, _trainPercent, _pathSave, _noiseLevel=0.):
        self.paraVecIndices = _paraVecIndices
        self.xparaVecIndices = _xparaVecIndices
        self.tProfBounds = _tProfBounds
        self.trainPercent = _trainPercent
        self.pathSave = _pathSave
        self.noiseLevel = _noiseLevel
        
    def visualizeData(self, picID, fontSize, Dict_baseline = {}, plotTProf=True, plotBaseline=False): 
        x_data, x_test, x_cv, y_data, y_test, y_cv, paraVec, xparaVec, rProf, startPoint, endPoint, indexer \
        = self.x_data, self.x_test, self.x_cv, self.y_data, self.y_test, self.y_cv, self.paraVec, self.xparaVec, \
            self.rProf, self.startPoint, self.endPoint, self.indexer    
                
        print('Added noise ' + str(self.noiseLevel))
        print("X_train shape: " + str(x_data.shape))
        print("Y_train shape: " + str(y_data.shape))
        print("X_test shape: " + str(x_test.shape))
        print("Y_test shape: " + str(y_test.shape))
        print("X_cv shape: " + str(x_cv.shape))
        print("Y_cv shape: " + str(y_cv.shape))
        print()
        
        alphaVal = 0.1
        tProfBoolean = False

        for ind in range(np.size(paraVec)):
            fig, ax = plt.subplots()
            n, bins, patches = ax.hist(y_data[:, ind], 100, density=1)
            ax.set_xlabel(paraVec[ind])
            fig = plt.figure(figsize=(14,4), dpi=200)
            nCols = np.size(xparaVec)
            nRows = 1
            for indX in range(nCols):
                ax = fig.add_subplot(nRows,nCols,indX+1)
                ax.plot(x_data[:,indX], y_data[:, ind],'ro', alpha=alphaVal)
                if plotBaseline:
                    ax.plot(x_data[:,indX],Dict_baseline['mu_var' + str(indX)] \
                                 +Dict_baseline['variance_var' + str(indX)],'k.')
                    ax.plot(x_data[:,indX],Dict_baseldine['mu_var' + str(indX)] \
                                 -Dict_baseline['variance_var' + str(indX)],'k.')
                ax.yaxis.labelpad = 0.8
                ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
                plt.title("train")
                plt.xlabel(xparaVec[indX])
                plt.ylabel(paraVec[ind])
            plt.tight_layout()
            plt.show()
            #fig.savefig(pathSave + "inverseProblemPics/" + "Input0_fig" + str(picID) + ".pdf", bbox_inches='tight')
            
        if  tProfBoolean and plotTProf:   
            length = int(np.size(paraVec))
            fig = plt.figure(figsize=(length,4), dpi=200)
            nCols = np.size(paraVec)
            nRows = 2
            for indY in range(nCols):
                vecInterest = y_data[:, indY]
                colorsVec= [plt.cm.jet(i) for i in vecInterest]
                ax = fig.add_subplot(nRows,nCols,indY+1)
                ax.set_prop_cycle('color',colorsVec)
                for i, ii in enumerate(indexer):
                    rProf = self.rProf[startPoint:endPoint] # profiles['Dict_Rprof_4p5Gyr' + str(ii)]
                    rProf = (rProf-rProf.min())/(rProf.max()-rProf.min())
                    tProf = x_data[i,np.size(xparaVec):]
                    ax.plot(tProf, rProf, linewidth=0.5)
                ax.yaxis.labelpad = 0.8
                ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
                if indY > 0:
                    ax.axes.get_yaxis().set_ticks([])
                ax.set_title(paraVec[indY])
                ax.set_xlabel("T") #[K]")
                ax.set_ylabel("R") #[km] ")
                for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                     ax.get_xticklabels() + ax.get_yticklabels()):
                    item.set_fontsize(12)
                #ax.set_xlim([250,2250])
                #ax.set_ylim([1700,3400])
            plt.tight_layout()
            plt.show()
           # fig.savefig(pathSave + "inverseProblemPics/" + "Input1_fig" + str(picID) + ".pdf", bbox_inches='tight')

In [58]:
class plotResults:
    def __init__(self, data, ax, ax1, alphaVal, title, _y_actual, Dict_prob_paras ,indPara,\
                 Dict_baseline, kernel, KMIX, plotIndividualPDFs=False):
        
        numParameters = len(data.paraVec)
        
        if "Test" in title:
            self.Dict_Test = Dict_prob_paras
        
        rho_m  = 3500. 
        g     = 3.7 
        alpha_m = 2.5e-5
        T_delta = 2000. 
        D = 1700e+3         
        k_diffusive = 1e-6 
        R = 8.314   
        
        def format_func(_val, tick_number):
            f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
            _g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
            fmt = mticker.FuncFormatter(_g)
            return "{}".format(fmt(_val))

        def dimensionalize(_val,_ind,isVariance=False):
            _min = data.pMin[_ind]
            _max = data.pMax[_ind]
            _val = _val*(_max-_min) 
            if not isVariance:
                _val = _val + _min
            
            if "Ra" in title and not isVariance:
                _val = np.log10(rho_m * g * alpha_m * T_delta * np.power(D,3.)/(np.power(10.,_val) * k_diffusive))
            
            if "ERef" in title:
                _val = _val*(R * T_delta)
            
            if "VRef" in title:
                _val = _val*(R * T_delta) /(rho_m * g * D)
            
            if "iniTempTop" in title:
                _val = _val*2000 
                if not isVariance:
                    _val = _val + 250
                    
            return _val

        xx = dimensionalize(_y_actual,indPara)    
        variance = []
        combined_mdn_std = []
        combined_mdn_mean = []
        xxSorted = np.sort(xx)        
        
        xxSize = np.size(xx)
        plotEvery = int(np.floor(xxSize*0.1))
        xxSorted.shape = (xxSize,1)
        Pr_xx = np.zeros((xxSize,xxSize))
        Pr_baseline = np.zeros((xxSize,xxSize))
        colors = ['r', 'g', 'm']
        
        def pdf(x):
            return 1./np.sqrt(2.*np.pi) * np.exp(-x**2/2.)
        #def cdf(x):
        #    return (1. + erf(x/np.sqrt(2))) / 2.
        def SSkew(x,a,e,w):
            t = (x-e) / w
            return 2. / w * pdf(t) * cdf(a*t)
        def getGamma(x):
            return m.gamma(x)
        def tf_beta(y, alpha, beta):
            Z = np.divide(np.multiply(getGamma(alpha),getGamma(beta)), getGamma(alpha + beta))
            result = np.divide(np.multiply(np.power(y,(alpha - 1.)), np.power((1. - y),(beta - 1.))),Z)
            return  result
             
        if plotIndividualPDFs:
            fig3 = plt.figure(figsize=(14,18))
           # plt.subplots_adjust(top=0.90, bottom=0.08, left=0.10, right=0.95, hspace=0.3,wspace=0.4)
            nRows = np.ceil((xxSize/plotEvery)/3)
            nCols = 3
            plotCounter = 1                    
                
        if kernel=='gaussian':
            for ind,val in enumerate(xxSorted):
                index = np.where(val == xx)[0][0]
                muIntermediate = []
                piIntermediate = []
                sigmaIntermediate = []
                for i in range(KMIX):
                    out_pi = np.asarray(Dict_prob_paras["multi_pi"+str(i)])[index]
                    out_sigma = np.asarray(Dict_prob_paras["multi_sigma_" + str(index) + "_" + str(i)])
                    out_cov = np.matmul(out_sigma,np.transpose(out_sigma))
                    out_mu = np.asarray(Dict_prob_paras["multi_mu"+str(i)])[index,:]
                    
                    muIntermediate.append(dimensionalize(out_mu[indPara],indPara))
                    sigmaIntermediate.append(dimensionalize(np.sqrt(out_cov[indPara,indPara]),indPara,True))
                    piIntermediate.append(out_pi)

                pr = piIntermediate[0] * norm.pdf(xxSorted, muIntermediate[0], sigmaIntermediate[0])
                for i in range(1,KMIX):
                    pr += piIntermediate[i] * norm.pdf(xxSorted, muIntermediate[i], sigmaIntermediate[i])
                pr.shape = (pr.shape[0],)
                
                mean_mdn = 0
                for i in range(KMIX):
                    mean_mdn += piIntermediate[i]*muIntermediate[i]
                var_mdn = 0
                #http://ecee.colorado.edu/~pao/anonftp/cdc02gaussmix.pdf: Equation 11
                for i in range(KMIX):
                    var_mdn += piIntermediate[i]*(np.power(sigmaIntermediate[i],2) + np.power(muIntermediate[i],2)) 
                var_mdn -= np.power(mean_mdn,2)
                
                combined_mdn_std.append(np.power(var_mdn,0.5))
                combined_mdn_mean.append(mean_mdn)
                
                Pr_xx[ind,:] = pr 
                if  plotIndividualPDFs and ind%plotEvery == 0:
                    ax3 = fig3.add_subplot(nRows,nCols,plotCounter) 
                    plotCounter += 1
                    legendStr = []
                    for i in range(np.size(muIntermediate)):
                        if piIntermediate[i] >= np.max(piIntermediate)*1e-15:
                            yPDF = piIntermediate[i] * norm.pdf(xxSorted, muIntermediate[i], sigmaIntermediate[i])
                            ax3.plot(xxSorted,yPDF,colors[i]+'--', linewidth=2.5)
                            legendStr.append("%.4f" % piIntermediate[i])
                            if "Test" in title:
                                self.Dict_Test["forFig_" + title + "_pdf_mixture" + str(i) + "_case_" + str(ind)] = yPDF
                    ax3.plot(xxSorted, Pr_xx[ind,:],"k-", linewidth=2.5)
                    ax3.plot([val, val],[0,max(Pr_xx[ind,:])], "-", color='grey', linewidth=2.5)
                    
                    if "Test" in title:
                        self.Dict_Test["forFig_" + title + "_trueVal_" + str(ind)] = val
                    
                    for item in ([ax3.title, ax3.xaxis.label, ax3.yaxis.label] +
                     ax3.get_xticklabels() + ax3.get_yticklabels()):
                        item.set_fontsize(20)
                    if "ERef" in title:
                        ax3.set_xticks([1e+5,3e+5,5e+5])
                        ax3.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                        ax3.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                    elif "VRef" in title:
                        ax3.set_xticks([4e-6, 7e-6, 10e-6])
                        ax3.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                        ax3.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                    plt.tight_layout()
        
        if "Test" in title:
            self.Dict_Test["forFig_" + title + "_xxSorted"] = xxSorted
            self.Dict_Test["forFig_" + title + "_Pr_xx"] = Pr_xx
        
        for ind,val in enumerate(xxSorted):
            index = np.where(val == xx)[0][0]
            muBase = dimensionalize(Dict_baseline["mu_var" + str(0) + "_" +  str(indPara)][index],indPara)
            sigmaBase = dimensionalize(Dict_baseline["variance_var" + str(0) + "_" +  str(indPara)][index],indPara,True)
            prb = 0
            prb = norm.pdf(xxSorted, muBase, sigmaBase)  
            prb.shape = (prb.shape[0],)
            Pr_baseline[ind,:] = prb 
            
        if "Test" in title:
            self.Dict_Test["forFig_" + title + "_averageSTD"] = np.mean(np.asarray(combined_mdn_std))            

        print("Average standard deviation: " + str(np.mean(np.asarray(combined_mdn_std))))
        
        if "Ra" in title:
            titlep = "$\log(\eta_{ref})$ [Pa s]"
        elif "ERef" in title:
            titlep = r"$E$ [J mol$^{-1}$]"
        elif "VRef" in title:
            titlep = r"$V$ [m$^3$ mol$^{-1}$]"
        elif "Enrichment_cr" in title:
            titlep = "$\Lambda$"
        elif "iniTempTop" in title:
            titlep = "$T_{ini}$ [K]"
        
        x = np.zeros((np.size(xxSorted),np.size(xxSorted)))
        y = np.zeros((np.size(xxSorted),np.size(xxSorted)))
        for ind,val in enumerate(xxSorted):
            x[ind,:] = val
            y[:,ind] = xxSorted[ind]
        ax.contourf(x,y,Pr_xx)
        
        if "Test" in title:
            self.Dict_Test["forFig_" + title + "x"] = x
            self.Dict_Test["forFig_" + title + "y"] = y
        
        ax.set_xlabel("True")
        ax.set_ylabel("Predicted")
        ax.set_title(titlep + "; MDN")
        
        ax1.contourf(x,y,Pr_baseline)  
        ax1.set_xlabel("True")
        ax1.set_ylabel("Predicted")
        ax1.set_title(titlep + "; MP")
        print()
        setTicks = True
        if setTicks:
            if "Ra" in title:
                ax.set_xticks([19,20,21,22])
                ax.set_yticks([19,20,21,22])
                ax1.set_xticks([19,20,21,22])
                ax1.set_yticks([19,20,21,22])

            if "ERef" in title:
                ax.set_xticks([1e+5,3e+5,5e+5])
                ax.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
                ax1.set_xticks([1e+5,3e+5,5e+5])
                ax1.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax1.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax1.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            elif "VRef" in title:
                ax.set_xticks([4e-6, 7e-6, 10e-6])
                ax.set_yticks([4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6, 10e-6])
                ax1.set_xticks([4e-6, 7e-6, 10e-6])
                ax1.set_yticks([4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6, 10e-6])
                ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax1.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                ax1.xaxis.set_major_formatter(plt.FuncFormatter(format_func))

            elif "Enrichment_cr" in title:
                ax.set_xticks([1,10,20,30,40,50])
                ax.set_yticks([1,10,20,30,40,50])
                ax1.set_xticks([1,10,20,30,40,50])
                ax1.set_yticks([1,10,20,30,40,50])
            elif "iniTempTop" in title:
                ax.set_xticks([1600,1650,1700,1750,1800])
                ax.set_yticks([1600,1650,1700,1750,1800])
                ax1.set_xticks([1600,1650,1700,1750,1800])
                ax1.set_yticks([1600,1650,1700,1750,1800])

            for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
                item.set_fontsize(20)

            for item in ([ax1.title, ax1.xaxis.label, ax1.yaxis.label] +
             ax1.get_xticklabels() + ax1.get_yticklabels()):
                item.set_fontsize(20)

In [59]:
class plotResults_multi:
    def __init__(self, data, Dict_prob_paras, KMIX, id_string):
        
        rho_m  = 3500. 
        g     = 3.7 
        alpha_m = 2.5e-5
        T_delta = 2000. 
        D = 1700e+3         
        k_diffusive = 1e-6 
        R = 8.314   
        
        numParameters = len(data.paraVec)
        plotParas = [r"$\log(\eta_{ref})$ [Pa s]", r"$E$ [J mol$^{-1}$]", r"$V$ [m$^3$ mol$^{-1}$]", 
                                     "$\Lambda$", "$T_{ini}$ [K]"]
        
        def format_func(_val, tick_number):
            f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
            _g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
            fmt = mticker.FuncFormatter(_g)
            return "{}".format(fmt(_val))

        def dimensionalize(_val,_ind,isVariance=False):
            _min = data.pMin[_ind]
            _max = data.pMax[_ind]
            _val = _val*(_max-_min) 
            if not isVariance:
                _val = _val + _min
            
            if _ind==0 and not isVariance:
                _val = np.log10(rho_m * g * alpha_m * T_delta * np.power(D,3.)/(np.power(10.,_val) * k_diffusive))
            if _ind==1:
                _val = _val*(R * T_delta)
            if _ind==2:
                _val = _val*(R * T_delta) /(rho_m * g * D)
            if _ind==4:
                _val = _val*2000 
                if not isVariance:
                    _val = _val + 250    
                    
            return _val
        
        def multivariate_gaussian(pos, _mu, Sigma):
            #https://scipython.com/blog/visualizing-the-bivariate-gaussian-distribution/
            """Return the multivariate Gaussian distribution on array pos.
            pos is an array constructed by packing the meshed arrays of variables
            x_1, x_2, x_3, ..., x_k into its _last_ dimension.

            """
            n = _mu.shape[0]
            Sigma_det = np.linalg.det(Sigma)
            Sigma_inv = np.linalg.inv(Sigma)
            N = np.sqrt((2*np.pi)**n * Sigma_det)
            # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
            # way across all the input variables.
            fac = np.einsum('...k,kl,...l->...', pos-_mu, Sigma_inv, pos-_mu)
            return np.exp(-fac / 2) / N
        
        figl = plt.figure(figsize=(15,3), dpi=320)
        pltcount = 1
        for paraInd in range(data.y_test.shape[1]):
            axl = figl.add_subplot(1,len(data.paraVec),pltcount)
            x = dimensionalize(data.y_test[:,paraInd],paraInd)
            rv = np.zeros((x.shape[0],1))
            for xind, xval in enumerate(x):
                rv[xind,0] = Dict_prob_paras["log_likelihood"][xind]
            axl.plot(x, rv, '.')
            axl.set_xlabel(plotParas[paraInd])
            if paraInd==0:
                axl.set_ylabel("Log-likelihood")
            axl.set_ylim([-2,16])
            setTicks = True
            if setTicks:
                axl.set_yticks([0,4,8,12,16])
                if paraInd==0:
                    axl.set_xticks([19,20,21,22])
                if paraInd==1:
                    axl.set_xticks([1e+5,3e+5,5e+5])
                    axl.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                if paraInd==2:
                    axl.set_xticks([4e-6, 7e-6, 10e-6])
                    axl.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                if paraInd==3:
                    axl.set_xticks([1,10,20,30,40,50])
                if paraInd==4:
                    axl.set_xticks([1600,1700,1800])
            
            for item in ([axl.title, axl.xaxis.label, axl.yaxis.label] +
             axl.get_xticklabels() + axl.get_yticklabels()):
                item.set_fontsize(18)
            pltcount += 1
        
        plt.tight_layout(w_pad=0.1)
        plt.show()
            
        for indInterest in [-1,9,14]: 
            figm = plt.figure(figsize=(14,12), dpi=320)
            pltcount = 1

            for ind_x, d_x in enumerate(data.paraVec):
                for ind_y, d_y in enumerate(data.paraVec):
                    if ind_x < ind_y:
                        ax2 = figm.add_subplot(len(data.paraVec),len(data.paraVec),pltcount)
                        if indInterest==-1:
                            x = dimensionalize(data.y_test[:,ind_x],ind_x)
                            y = dimensionalize(data.y_test[:,ind_y],ind_y)
                            z = Dict_prob_paras["log_likelihood"]
                            ax2.scatter(x, y, c=z, s=50, cmap="Greys")
                            ax2.scatter(x[9], y[9], s=80, facecolors='none', edgecolors='b', linewidth=2.5)
                            ax2.scatter(x[14], y[14], s=80, facecolors='none', edgecolors='r', linewidth=2.5)
                            ax2.plot(x, y, color='none')
                            ax2.relim()
                            ax2.autoscale_view()
                        else:
                            actVal_x = dimensionalize(data.y_test[indInterest,ind_x],ind_x)
                            actVal_y = dimensionalize(data.y_test[indInterest,ind_y],ind_y)
                            x = dimensionalize(np.linspace(0,1,1000),ind_x)
                            y = dimensionalize(np.linspace(0,1,1000),ind_y)
                            x_, y_ = np.meshgrid(x, y)
                            pos = np.dstack((x_, y_))
                            rv=0
                            for i in range(KMIX):
                                out_pi = np.asarray(Dict_prob_paras["multi_pi"+str(i)])[indInterest]
                                out_sigma = np.asarray(Dict_prob_paras["multi_sigma_" + str(indInterest) + "_" + str(i)])
                                out_mu = np.asarray(Dict_prob_paras["multi_mu"+str(i)])[indInterest,:]
                                out_cov = np.matmul(out_sigma, np.transpose(out_sigma))
                                cov_matrix = [[out_cov[ind_x,ind_x],out_cov[ind_x,ind_y]], 
                                              [out_cov[ind_y,ind_x],out_cov[ind_y,ind_y]]]  # scale matrix from diag

                                dim_x = data.pMax[ind_x]-data.pMin[ind_x]
                                dim_y = data.pMax[ind_y]-data.pMin[ind_y]                                

                                if ind_x==1:
                                    dim_x *= (R * T_delta)
                                if ind_x==2:
                                    dim_x *= (R * T_delta) /(rho_m * g * D)
                                if ind_x==4:
                                    dim_x *= 2000
                                if ind_y==1:
                                    dim_y *= (R * T_delta)
                                if ind_y==2:
                                    dim_y *= (R * T_delta) /(rho_m * g * D)
                                if ind_y==4:
                                    dim_y *= 2000 

                                cov_matrix[0][0] *= dim_x*dim_x 
                                cov_matrix[0][1] *= dim_x*dim_y
                                cov_matrix[1][0] *= dim_y*dim_x 
                                cov_matrix[1][1] *= dim_y*dim_y

                                rv += out_pi*multivariate_gaussian(pos,np.asarray([dimensionalize(out_mu[ind_x],ind_x),
                                                                  dimensionalize(out_mu[ind_y],ind_y)]), 
                                                                 np.asarray(cov_matrix))
                            ax2.contour(x_, y_, rv)
                            if indInterest==9:
                                colorUse = "b"
                            else:
                                colorUse = "r"
                            ax2.plot(actVal_x,actVal_y,colorUse+'X',markersize=9)
                        ax2.set_xlabel(plotParas[ind_x])
                        ax2.set_ylabel(plotParas[ind_y])
                        setTicks = True
                        if setTicks:
                            if ind_x==0:
                                ax2.set_xticks([19,20,21,22])
                            if ind_y==0:
                                ax2.set_yticks([19,20,21,22])
                            if ind_x==1:
                                ax2.set_xticks([1e+5,3e+5,5e+5])
                                ax2.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                            if ind_y==1:
                                ax2.set_yticks([1e+5,3e+5,5e+5])
                                ax2.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                            if ind_x==2:
                                ax2.set_xticks([4e-6, 7e-6, 10e-6])
                                ax2.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                            if ind_y==2:
                                ax2.set_yticks([4e-6, 7e-6, 10e-6])
                                ax2.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                            if ind_x==3:
                                ax2.set_xticks([1,10,20,30,40,50])
                            if ind_y==3:
                                ax2.set_yticks([1,10,20,30,40,50])
                            if ind_x==4:
                                ax2.set_xticks([1600,1650,1700,1750,1800])
                            if ind_y==4:
                                ax2.set_yticks([1600,1650,1700,1750,1800])

                            for item in ([ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
                             ax2.get_xticklabels() + ax2.get_yticklabels()):
                                item.set_fontsize(15)
                    pltcount += 1
                        
            plt.tight_layout(w_pad=0.1)
            plt.show()
            
            if indInterest==-1:
                vecInterest = Dict_prob_paras["log_likelihood"]
                vecInterestTemp = np.asarray(vecInterest)
                vecInterest = np.asarray(vecInterest)
                vecInterest = np.interp(vecInterest, (vecInterest.min(), vecInterest.max()), (0, 1))

                figc, axc = plt.subplots(figsize=(6,0.2), dpi=320)
                gradient= np.sort(vecInterest)
                gradient = np.vstack((gradient, gradient))
                axc.imshow(gradient, aspect='auto', cmap=plt.get_cmap('Greys'))
                axc.set_yticklabels([])
                axc.set_yticks([])
                #ax.set_xscale('log')

                my_xticks = axc.get_xticks()
                plt.xticks([my_xticks[1], my_xticks[-2]],['{:.1f}'.format(min(vecInterestTemp)), '{:.1f}'.format(max(vecInterestTemp))]
                           ,visible=True, rotation="horizontal")
                plt.xlabel('Log-likelihood')

            #picDirectory = pathSave + "Pics/" + id_string + "/"
            #if not os.path.exists(picDirectory):
            #    os.makedirs(picDirectory)
            #figm.savefig(picDirectory + "multiVar_" + str(indInterest) + ".pdf", bbox_inches='tight')

In [60]:
class MDN_K:
    def __init__(self, data, x_data,y_data, x_test,y_test, x_cv,y_cv, hSize, KMIX, NEPOCH, learnRate, \
                 paraVec, xparaVec, Dict_baseline_train, Dict_baseline_test, picID, repeats, id_string, \
                 kernel='gaussian',activationFunc='tanh',trainOrload='load'):
        
        print("Hidden Layers: " + str(hSize))
        print("Number of Mixtures: " + str(KMIX))
        print('Kernel is ' + str(kernel))
        print('Activation is ' + str(activationFunc))
        print()
            
        print("Training data: " + str(x_data.shape) + "," + str(y_data.shape))
        print("Test data: " + str(x_test.shape) + "," + str(y_test.shape))
        print("CV data: " + str(x_cv.shape) + "," + str(y_cv.shape))
        print()
        
        yNumParameters = y_data.shape[1] 
        graph = tf.get_default_graph()        
        l2reg = keras.regularizers.l2(0.00001/x_data.shape[0])
        xSize = x_data.shape[1]
        
        model = keras.Sequential()
        model.add(keras.layers.Dense(hSize[0], input_dim=(xSize), activation=activationFunc, kernel_regularizer=l2reg, dtype="float64"))
        for h in hSize[1:]:
            model.add(keras.layers.Dense(h, activation=activationFunc, kernel_regularizer=l2reg, dtype="float64"))
        model.add(mdn.MDN(yNumParameters, KMIX))

        class TimeHistory(keras.callbacks.Callback):
            def on_train_begin(self, logs={}):
                self.times = []
            def on_epoch_begin(self, batch, logs={}):
                self.epoch_time_start = time.time()
            def on_epoch_end(self, batch, logs={}):
                self.times.append(time.time() - self.epoch_time_start)

        config = tf.ConfigProto(
        intra_op_parallelism_threads=1,
        allow_soft_placement=True)

        config.gpu_options.allow_growth = True
        config.gpu_options.per_process_gpu_memory_fraction = 0.8

        session = tf.Session(config=config)
        name = id_string + "_multiVariate"
        with session.as_default():
            with session.graph.as_default():
                pathSaveK = pathSave + "/Data_Files/"
                if trainOrload=='load':
                    model =  keras.models.load_model(pathSaveK + name + ".hdf5",\
                            custom_objects={'MDN': mdn.MDN, 'mdn_loss_func': mdn.get_mixture_loss_func(yNumParameters, KMIX)})
                    model.summary()
                    
                else:
                    model.compile(loss=mdn.get_mixture_loss_func(yNumParameters,KMIX), optimizer=keras.optimizers.Adam(lr=learnRate))
                    model.summary()
                    
                    cp = keras.callbacks.ModelCheckpoint(pathSaveK + name +'.hdf5', \
                         monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
                    csvlogger = tf.keras.callbacks.CSVLogger(pathSaveK + name + '.txt', separator=",", append=False)

                    time_callback = TimeHistory()
                    history = History()
                    history = model.fit(x_data, y_data,
                        verbose=0,
                        epochs=NEPOCH,
                        batch_size=32,
                        shuffle=True,
                        validation_data=(x_cv, y_cv),
                        callbacks=[time_callback, cp, keras.callbacks.TerminateOnNaN(), csvlogger])
                        
                f = open(pathSaveK + name + '.txt', 'r')
                lines = f.readlines()                    
                loss = []
                loss_cv = []                    
                for _l in lines[1:]:
                    loss.append(float(_l.split(',')[1]))
                    loss_cv.append(float(_l.split(',')[2]))
                    
                y_trainML = model.predict(x_data)
                y_testML = model.predict(x_test)
                
                Dict_Train = {}
                Dict_Test = {}
                Dict_CV = {}

                def generateDict(x_I, yNumParameters):
                    Dict_I = {}
                    sigNum = int(yNumParameters*(yNumParameters-1)/2 +yNumParameters)
                    mus = np.apply_along_axis((lambda a: a[:KMIX*yNumParameters]), 1, x_I)
                    sigs = np.apply_along_axis((lambda a: a[KMIX*yNumParameters:KMIX*yNumParameters+KMIX*sigNum]), 1, x_I)
                    pis = np.apply_along_axis((lambda a: a[-KMIX:]), 1, x_I)
                    for piInd in range(pis.shape[0]):
                        pis[piInd,:] = mdn.softmax(pis[piInd,:])
                    
                    for i in range(KMIX):
                        for ii in range(x_I.shape[0]):
                            sig_vals = sigs[ii,i*sigNum:(i+1)*sigNum]
                            sig_vals_a = np.concatenate([sig_vals, sig_vals[yNumParameters:][::-1]])
                            sig_vals_b = np.reshape(sig_vals_a[::-1], [yNumParameters,yNumParameters])
                            sig_vals_c = np.tril(sig_vals_b, k=0)
                            for r in range(yNumParameters):
                                for c in range(yNumParameters):
                                    if r==c:
                                        sig_vals_c[r,c] = np.exp(sig_vals_c[r,c])
                            Dict_I["multi_sigma_" + str(ii) + "_" + str(i)] = sig_vals_c
                            
                        Dict_I["multi_mu" + str(i)] = mus[:,i*yNumParameters:(i+1)*yNumParameters]
                        Dict_I["multi_pi" + str(i)] = pis[:,i]
                        
                    Dict_I["log_likelihood"] = mdn.get_mixture_log_likelihood(y_test,y_testML,KMIX,yNumParameters).eval()
                    return Dict_I
                
                Dict_Train = generateDict(y_trainML, yNumParameters)
                Dict_Test = generateDict(y_testML, yNumParameters)
        
                alphaVal = 0.5
                
                plotResults_multi(data, Dict_Test, KMIX, id_string)
                
                plotIndividual = True
                if plotIndividual:
                    for ind in range(yNumParameters):
                        fig = plt.figure(figsize=(14,9), dpi=1000)
                        nCols = 3 
                        nRows = 2 
                        plotCounter = 1        

                        ax = fig.add_subplot(nRows,nCols,plotCounter) 
                        ax.plot(loss, 'r-', label='Training')
                        ax.plot(loss,"b--", label='Validation')
                        plt.title('Loss')
                        ax.set_xlabel('Epochs')
                        ax.set_ylabel('Negative log-likelihood')
                        ax.legend()
                        #ax.locator_params(axis='x', nbins=2)
                        plotCounter += 1
                        ax = fig.add_subplot(nRows,nCols,plotCounter)
                        plotCounter += 1
                        ax1 = fig.add_subplot(nRows,nCols,plotCounter)
                        plotCounter += 2

                        strhere = paraVec[ind]
                        pR = plotResults(data,ax,ax1, alphaVal,(paraVec[ind] + ", Train"), y_data[:,ind], Dict_Train, \
                                     ind, Dict_baseline_train, kernel, KMIX, False)

                        ax = fig.add_subplot(nRows,nCols,plotCounter)
                        plotCounter += 1
                        ax1 = fig.add_subplot(nRows,nCols,plotCounter)
                        plotCounter += 1
                        pRT = plotResults(data,ax,ax1, alphaVal,(paraVec[ind] + ", Test"), y_test[:,ind], Dict_Test, \
                                ind, Dict_baseline_test, kernel, KMIX, False)
                        plt.tight_layout()
                        plt.show()
                        
                #with open(pathSave + "/resultsDicts/multiVar_noise_dict" + str(data.noiseLevel) + ".txt", "wb") as fkl:
                #    dump(pRT.Dict_Test, fkl)
                    
                print("-------------------------------------------------------------------------------------------------------")
                print("-------------------------------------------------------------------------------------------------------")
                print("-------------------------------------------------------------------------------------------------------")
                print()

In [61]:
class meanPredictor:
    def __init__(self, x_data,y_data,xparaVec, plotMP=False):
        Dict_Mean_Predictor = {}
        if 'Tprof_4p5Gyr' in xparaVec and np.size(xparaVec)==1:
            tProfOnly = True
        else:
            tProfOnly = False
        for i in range(np.size(xparaVec)):
            for j in range(y_data.shape[1]):
                if tProfOnly or (xparaVec[i] == 'Tprof_4p5Gyr'):
                    observable = np.mean(x_data,axis=1)
                else:
                    observable = x_data[:,i] 
                para = y_data[:,j]
                steps = 10

                bin_means, bin_edges, binnumber = binned_statistic(observable, para, statistic='mean',bins=steps)
                bin_variance = [np.std(para[np.where(binnumber == bIndex)[0]]) for bIndex in range(1,steps+1)]
                removeVec = []
                notRemoveVec = []
                for i3 in range(np.size(bin_means)):
                    if np.isnan(bin_means[i3]) or bin_variance[i3] == 0.0:
                        removeVec.append(i3)
                    else:
                        notRemoveVec.append(i3)

                for r in removeVec:
                    bin_variance[r] = np.average([bin_variance[notInd] for notInd in notRemoveVec])
                    bin_means[r] = np.average([bin_means[notInd] for notInd in notRemoveVec])

                digitizer = np.digitize(observable,bin_edges) - 1
                mu = np.zeros(np.size(digitizer))
                variance = np.zeros(np.size(digitizer))
                for ind in range(np.size(digitizer)):
                    mu[ind] = bin_means[digitizer[ind]-1]
                    variance[ind] = bin_variance[digitizer[ind]-1]
                Dict_Mean_Predictor["mu_var" + str(i) + "_" + str(j)] = mu
                Dict_Mean_Predictor["variance_var" + str(i) + "_" + str(j)] = variance
                if plotMP:
                    plt.figure(figsize=(4,6))
                    plt.plot(observable,para, "ro", alpha = 0.4)
                    plt.plot((bin_edges[1:]+bin_edges[0:-1])/2.,bin_means, "k-")
                    plt.plot((bin_edges[1:]+bin_edges[0:-1])/2.,bin_means+bin_variance,"b--")
                    plt.plot((bin_edges[1:]+bin_edges[0:-1])/2.,bin_means-bin_variance,"b--")
                    plt.legend([_, "$\mu$", "$\mu + \sigma$", "$\mu - \sigma$"], loc="lower_right")
        self.Dict_Mean_Predictor = Dict_Mean_Predictor

In [62]:
def bigLoop(_zipped, repeats=1, trainOrload='load'):
    Dict_kls = {}
    hSize, KMIX, x_o, x_b, trainPercent, yIndex, kernel, activation, noiseLevel = _zipped
    
    fontSize = 14
    picID = 0 #yIndices  [yIndices[i5]]
    data = getData([yIndex], x_o, x_b, trainPercent, pathSave, noiseLevel)

    id_string = str(_zipped)

    with open(pathSave + "/Data_Files/processedData" + id_string + ".txt", "rb") as fkl:
        dataDict = load(fkl)

    data.x_data = dataDict['x_data']
    data.x_test = dataDict['x_test']
    data.x_cv = dataDict['x_cv']     
    data.y_data = dataDict['y_data']
    data.y_test = dataDict['y_test']
    data.y_cv = dataDict['y_cv']     
    data.paraVec = dataDict['paraVec']
    data.xparaVec = dataDict['xparaVec']
    data.rProf = dataDict['rProf']
    data.startPoint = dataDict['startPoint']
    data.endPoint = dataDict['endPoint']    
    data.indexer = dataDict['indexer']    
    data.pMax = dataDict['pMax']  
    data.pMin = dataDict['pMin']  
    data.oMax = dataDict['oMax']
    data.oMin = dataDict['oMin']

    Dict_MP_Test = meanPredictor(data.x_test, data.y_test, data.xparaVec, False)
    Dict_MP_Train = meanPredictor(data.x_data, data.y_data, data.xparaVec, False)

    #data.visualizeData(data, picID, fontSize, Dict_MP_Train.Dict_Mean_Predictor)

    MDN_K(data,data.x_data,data.y_data, data.x_test,data.y_test, data.x_cv, \
          data.y_cv, hSize, KMIX, 50000, 0.00001, \
          data.paraVec, data.xparaVec, Dict_MP_Train.Dict_Mean_Predictor, \
          Dict_MP_Test.Dict_Mean_Predictor, picID, repeats, id_string, kernel, activation, trainOrload)
    picID += 1

In [ ]:
zipped_new = [[[60, 60, 60], 3, [0,1,2,3,4,5,6], [0, 100], 0.9, [0, 4, 5, 6, 7], 'gaussian', 'tanh', 0.0]]

runParallel = False
trainOrload = 'load'
repeats = 1

if runParallel:                   
    Parallel(n_jobs=1, verbose=10, backend='loky', prefer='processes')(delayed(bigLoop)(_z, repeats, trainOrload) for _z in zipped_new)
else:
    [bigLoop(_z, repeats, trainOrload) for _z in zipped_new] 